In [1]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("ExemploAmostraDados").getOrCreate()

Para converter o JSON fornecido para um arquivo CSV usando PySpark, você pode seguir estes passos:

1. Instale PySpark, se ainda não estiver instalado.
2. Crie um script PySpark para ler o JSON, processar os dados e salvá-los em um arquivo CSV.

In [3]:
import json

f = open("../data/Localidades_Brasil.json", encoding='utf8')

json_data = json.load(f)
print(json_data)

{'Brasil': {'estados': [{'nome': 'São Paulo', 'cidades': [{'nome': 'Cotia', 'latitude': -23.6025, 'longitude': -46.9191, 'bairros': [{'nome': 'Centro', 'latitude': -23.603, 'longitude': -46.9175}, {'nome': 'Granja Viana', 'latitude': -23.5929, 'longitude': -46.8414}]}, {'nome': 'São Paulo', 'latitude': -23.5505, 'longitude': -46.6333, 'bairros': [{'nome': 'Vila Mariana', 'latitude': -23.5894, 'longitude': -46.6342}, {'nome': 'Pinheiros', 'latitude': -23.5614, 'longitude': -46.6996}]}]}, {'nome': 'Rio de Janeiro', 'cidades': [{'nome': 'Rio de Janeiro', 'latitude': -22.9068, 'longitude': -43.1729, 'bairros': [{'nome': 'Copacabana', 'latitude': -22.9711, 'longitude': -43.1822}, {'nome': 'Ipanema', 'latitude': -22.9836, 'longitude': -43.2075}]}]}]}}


In [6]:
df = spark.read.json(spark.sparkContext.parallelize([json_data]))
df.printSchema()

root
 |-- Brasil: struct (nullable = true)
 |    |-- estados: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cidades: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- bairros: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |    |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |    |    |    |    |-- nome: string (nullable = true)
 |    |    |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- nome: string (nullable = true)



In [8]:
df_estados = df.select(explode("Brasil.estados").alias("estado"))
df_estados.printSchema()

root
 |-- estado: struct (nullable = true)
 |    |-- cidades: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- bairros: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |-- nome: string (nullable = true)



In [9]:
df_cidades = df_estados.select(col("estado.nome").alias("estado_nome"), explode("estado.cidades").alias("cidade"))
df_cidades.printSchema()

root
 |-- estado_nome: string (nullable = true)
 |-- cidade: struct (nullable = true)
 |    |-- bairros: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- nome: string (nullable = true)



In [11]:
df_bairros = df_cidades.select(
    col("estado_nome"),
    col("cidade.nome").alias("cidade_nome"),
    col("cidade.latitude").alias("cidade_latitude"),
    col("cidade.longitude").alias("cidade_longitude"),
    explode("cidade.bairros").alias("bairro")
)
df_bairros.printSchema()

root
 |-- estado_nome: string (nullable = true)
 |-- cidade_nome: string (nullable = true)
 |-- cidade_latitude: double (nullable = true)
 |-- cidade_longitude: double (nullable = true)
 |-- bairro: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- nome: string (nullable = true)



In [15]:
df_final = df_bairros.select(
    col("estado_nome"),
    col("cidade_nome"),
    col("cidade_latitude"),
    col("cidade_longitude"),
    col("bairro.nome").alias("bairro_nome"),
    col("bairro.latitude").alias("bairro_latitude"),
    col("bairro.longitude").alias("bairro_longitude")
)

df_final.show()

+--------------+--------------+---------------+----------------+------------+---------------+----------------+
|   estado_nome|   cidade_nome|cidade_latitude|cidade_longitude| bairro_nome|bairro_latitude|bairro_longitude|
+--------------+--------------+---------------+----------------+------------+---------------+----------------+
|     São Paulo|         Cotia|       -23.6025|        -46.9191|      Centro|        -23.603|        -46.9175|
|     São Paulo|         Cotia|       -23.6025|        -46.9191|Granja Viana|       -23.5929|        -46.8414|
|     São Paulo|     São Paulo|       -23.5505|        -46.6333|Vila Mariana|       -23.5894|        -46.6342|
|     São Paulo|     São Paulo|       -23.5505|        -46.6333|   Pinheiros|       -23.5614|        -46.6996|
|Rio de Janeiro|Rio de Janeiro|       -22.9068|        -43.1729|  Copacabana|       -22.9711|        -43.1822|
|Rio de Janeiro|Rio de Janeiro|       -22.9068|        -43.1729|     Ipanema|       -22.9836|        -43.2075|
+